## 数据对比
看看

In [ ]:
import numpy as np
import pandas as pd
from pyhive import presto

presto_conn = presto.connect(
    host = 'emr-bi',
    port = 9090,
    catalog='hive',
    schema='default'
)

conn = presto_conn.cursor()
hql = '''
    select
        dt,
        app_version,
        count(distinct device_id) as dau1,
        count(distinct device_id2) as dau2,
        count(distinct if(is_new_device2=-1, device_id, null)) as u3,
        count(distinct if(is_new_device2=-1 and launch_type=1, device_id, null)) as u4
    from 
        (select
            dt,
            device_id,
            device_id2,
            is_new_device2,
            launch_type,
            (case 
                when app_version>='9.2.4' then '9.2.4+'
                when app_version is null or app_version='' then ''
                when app_version<'9.2.4' then '9.2.4-'
                else app_version
            end) as app_version
        from bi_dwm.dwm_xinai_device_info_dd
        where dt>='2022-03-01' and dt<='2022-03-15' and plat='app' and os='Android'
        ) as a
    where a.app_version!=''
    group by dt, app_version
    order by dt
'''
conn.execute(hql)
data = conn.fetchall()
df = pd.DataFrame(np.random.randn(10, 2)*100)
df.columns = ['日期', '版本', '新DAU', '旧DAU', '', '', '', '']